## Tensorflow基本操作

In [ ]:
import tensorflow as tf
# 定义一个随机数（标量）
random_float = tf.random.uniform(shape=())

# 定义一个有2个元素的零向量
zero_vector = tf.zeros(shape=(2))

# 定义两个2×2的常量矩阵
A = tf.constant([[1., 2.], [3., 4.]])
B = tf.constant([[5., 6.], [7., 8.]])

# 查看矩阵A的形状、类型和值
print(A.shape)      # 输出(2, 2)，即矩阵的长和宽均为2
print(A.dtype)      # 输出<dtype: 'float32'>
print(A.numpy())    # 输出[[1. 2.]
                    #      [3. 4.]]

In [ ]:
C = tf.add(A, B)    # 计算矩阵A和B的和
D = tf.matmul(A, B) # 计算矩阵A和B的乘积
print(C)
print(D)

## 自动求导机制

TensorFlow 提供了强大的 自动求导机制 来计算导数。在即时执行模式下，TensorFlow 引入了 tf.GradientTape() 这个 “求导记录器” 来实现自动求导。以下代码展示了如何使用 tf.GradientTape() 计算函数 $y(x) = x^2$ 在 $x = 3$ 时的导数：

In [ ]:
import tensorflow as tf

x = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:     # 在 tf.GradientTape() 的上下文内，所有计算步骤都会被记录以用于求导
    y = tf.square(x)
y_grad = tape.gradient(y, x)        # 计算y关于x的导数
print(y, y_grad)

在机器学习中，更加常见的是对多元函数求偏导数，以及对向量或矩阵的求导。下面使用 tf.GradientTape() 计算函数 $L(w, b) = \|Xw + b - y\|^2$ 在 $w = (1, 2)^T, b = 1$ 时分别对 $w, b$ 的偏导数。其中 $X = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix},  y = \begin{bmatrix} 1 \\ 2\end{bmatrix}$。

In [ ]:
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
    L = tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])        # 计算L(w, b)关于w, b的偏导数
print(L, w_grad, b_grad)

## 线性模型

对于多元函数 $f(x)$ 求局部极小值，梯度下降的过程如下：

初始化自变量为 $x_0 ， k=0$

迭代进行下列步骤直到满足收敛条件：

- 求函数 $f(x)$ 关于自变量的梯度 $\nabla f(x_k)$

- 更新自变量： $x_{k+1} = x_{k} - \gamma \nabla f(x_k)$ 。这里 $\gamma$ 是学习率（也就是梯度下降一次迈出的 “步子” 大小）

- $k \leftarrow k+1$

接下来，我们考虑如何使用程序来实现梯度下降方法，求得线性回归的解 $\min_{a, b} L(a, b) = \sum_{i=1}^n(ax_i + b - y_i)^2$ 。

### 使用Tensorflow API

In [ ]:
import numpy as np

X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

In [ ]:
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(num_epoch):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        # 对所有元素求和
        loss = tf.reduce_sum(tf.square(y_pred - y))
    # TensorFlow自动计算损失函数关于自变量（模型参数）的梯度
    grads = tape.gradient(loss, variables)
    # TensorFlow自动根据梯度更新参数
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))
print(variables)

### 使用keras API

**模型（Model）与层（Layer）**

在 TensorFlow 中，推荐使用 Keras（ tf.keras ）构建模型。Keras 是一个广为流行的高级神经网络 API，简单、快速而不失灵活性，现已得到 TensorFlow 的官方内置和全面支持。

Keras 有两个重要的概念： 模型（Model） 和 层（Layer） 。层将各种计算流程和变量进行了封装（例如基本的全连接层，CNN 的卷积层、池化层等），而模型则将各种层进行组织和连接，并封装成一个整体，描述了如何将输入数据通过各种层以及运算而得到输出。在需要模型调用的时候，使用 y_pred = model(X) 的形式即可。Keras 在 tf.keras.layers 下内置了深度学习中大量常用的的预定义层，同时也允许我们自定义层。

![titile](keras_model.png)

In [ ]:
import tensorflow as tf
# 根据X的维度可知，这是一个三元线性回归
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])

# 定义model Linear
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )

    def call(self, input):
        output = self.dense(input)
        return output


model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)      # 调用模型 y_pred = model(X) 而不是显式写出 y_pred = a * X + b
        loss = tf.reduce_mean(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)    # 使用 model.variables 这一属性直接获得模型中的所有变量
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

全连接层 （Fully-connected Layer，tf.keras.layers.Dense ）是 Keras 中最基础和常用的层之一，对输入矩阵 A 进行 $f(AW + b)$ 的线性变换 + 激活函数操作。如果不指定激活函数，即是纯粹的线性变换$AW + b$。具体而言，给定输入张量 input = [batch_size, input_dim] ，该层对输入张量首先进行 tf.matmul(input, kernel) + bias 的线性变换（ kernel 和 bias 是层中可训练的变量），然后对线性变换后张量的每个元素通过激活函数 activation ，从而输出形状为 [batch_size, units] 的二维张量。

![titile](dense.png)

其包含的主要参数如下：

- units ：输出张量的维度；

- activation ：激活函数，对应于 f(AW + b) 中的 f ，默认为无激活函数（ a(x) = x ）。常用的激活函数包括 tf.nn.relu 、 tf.nn.tanh 和 tf.nn.sigmoid ；

- use_bias ：是否加入偏置向量 bias ，即 f(AW + b) 中的 b。默认为 True ；

- kernel_initializer 、 bias_initializer ：权重矩阵 kernel 和偏置向量 bias 两个变量的初始化器。默认为 tf.glorot_uniform_initializer。设置为 tf.zeros_initializer 表示将两个变量均初始化为全 0；

该层包含权重矩阵 kernel = [input_dim, units] 和偏置向量 bias = [units] 2 两个可训练变量，对应于 f(AW + b) 中的 W 和 b。

## 卷积神经网络CNN

卷积神经网络 （Convolutional Neural Network, CNN）是一种结构类似于人类或动物的 视觉系统 的人工神经网络，包含一个或多个卷积层（Convolutional Layer）、池化层（Pooling Layer）和全连接层（Fully-connected Layer）。

imdb数据：http://ai.stanford.edu/~amaas/data/sentiment/

### 定义模型

在 tf.keras.layers.Conv2D 中，当我们将 padding 参数设为 same 时，会将周围缺少的部分使用 0 补齐，使得输出的矩阵大小和输入一致。

通过 tf.keras.layers.Conv2D 的 strides 参数即可设置步长（默认为 1）。

**tf.keras.models.Sequential()**

最典型和常用的神经网络结构是将一堆层按特定顺序叠加起来，使用tf.keras.models.Sequential()，我们只需要提供一个层的列表，就能由 Keras 将它们自动首尾相连，形成模型。

**tf.keras.Model.compile**

当模型建立完成后，通过 tf.keras.Model 的 compile 方法配置训练过程：

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )
tf.keras.Model.compile 接受 3 个重要的参数：

- oplimizer ：优化器，可从 tf.keras.optimizers 中选择；

- loss ：损失函数，可从 tf.keras.losses 中选择；

- metrics ：评估指标，可从 tf.keras.metrics 中选择

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 300
sequence_length = 300
embedding_dimension = 100

# 生成数据
def gen_datasets():
    '''
    generate train datasets and testing datasets from imdb data
    :return:
        x_train, y_train,
        x_test, y_test
    '''
    (x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=num_words)
    return  (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = gen_datasets()
x_train = pad_sequences(x_train, maxlen=sequence_length) 
x_test = pad_sequences(x_test,maxlen=sequence_length)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

# 构造分类器
filter_sizes = [3, 4, 5]
def convolution():
    # 定义输入层，制定输入数据的形状
    inn = layers.Input(shape=(sequence_length, embedding_dimension, 1))
    cnns = []
    for size in filter_sizes:
        conv = layers.Conv2D(filters=64,                                # 卷积层神经元（卷积核）数目
                             kernel_size=(size, embedding_dimension),   # 卷积核大小
                             strides=1,                                 # 设置步长
                             padding='valid',                           # padding策略（vaild 或 same） 
                             activation='relu')(inn)                    # 激活函数
        
        pool = layers.MaxPool2D(pool_size=(sequence_length - size + 1, 1), 
                                           padding='valid')(conv)
        cnns.append(pool)
    outt = layers.concatenate(cnns)

    model = keras.Model(inputs=inn, outputs=outt)
    return model


def cnn_mulfilter():
    model = keras.Sequential([
        layers.Embedding(input_dim=num_words, output_dim=embedding_dimension,
                         input_length=sequence_length),
        layers.Reshape((sequence_length, embedding_dimension, 1)),
        convolution(),
        layers.Flatten(),
        layers.Dense(10, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')

    ])
    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model


model = cnn_mulfilter()
model.summary()

### 训练

**f.keras.Model.fit**

model.fit(data_loader.train_data, 
       data_loader.train_label, 
       epochs=num_epochs, 
       batch_size=batch_size)

f.keras.Model.fit 接受 5 个重要的参数：

- x ：训练数据；

- y ：目标数据（数据标签）；

- epochs ：将训练数据迭代多少遍；

- batch_size ：批次的大小；

- validation_data ：验证数据，可用于在训练过程中监控模型的性能。

In [ ]:
history = model.fit(x_train, y_train, batch_size=64, epochs=5, validation_split=0.1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'valiation'], loc='upper left')
plt.show()

### 测试

使用 tf.keras.Model.evaluate 评估训练效果，提供测试数据及标签即可

In [ ]:
result = model.evaluate(x_test, y_test)
print("loss=%s,accuracy=%s" %(result[0],result[1]))